# Статистический анализ данных сервиса аренды самокатов

- **Описание**: Цель проекта — провести статистический анализ данных пользователей и их поездок для сервиса аренды самокатов. В рамках исследования анализируются данные о подписках, поездках и пользователях из разных городов. Основной задачей является выявление паттернов поведения пользователей и проверка гипотез, чтобы помочь бизнесу повысить доходность и оптимизировать маркетинговые стратегии.
- **Задачи исследования**:
    1. Исследовать демографические данные пользователей и их поведенческие паттерны.
    2. Сравнить пользователей с подпиской и без подписки по ключевым показателям (выручка, время и расстояние поездок).
    3. Оценить сезонные колебания выручки для оптимизации маркетинговых активностей.
    4. Проверить гипотезы о поведении пользователей с подпиской и без подписки.
    5. Определить оптимальные параметры для рассылок промокодов и push-уведомлений.

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
from scipy.stats import binom
from scipy.stats import norm

## Загрузка данных

- Загрузим каждый CSV-файл в отдельный датафрейм, используя библиотеку pandas.
- Выведем первые строки каждого датафрейма для знакомства с данными.
- Получим общую информацию о каждом датафрейме, чтобы понять структуру и типы данных.

**Описание данных**:

В основных данных есть информация о пользователях, их поездках и подписках.

- **Пользователи — `users_go.csv`**:
    - `user_id` — уникальный идентификатор пользователя.
    - `name` — имя пользователя.
    - `age` — возраст пользователя.
    - `city` — город проживания пользователя.
    - `subscription_type` — тип подписки (free или ultra).
- **Поездки — `rides_go.csv`**:
    - `user_id` — уникальный идентификатор пользователя.
    - `distance` — расстояние поездки (в метрах).
    - `duration` — продолжительность поездки (в минутах).
    - `date` — дата поездки.
- **Подписки** — **`subscriptions_go.csv`**:
    - `subscription_type` — тип подписки.
    - `minute_price` — стоимость одной минуты поездки для данного типа подписки.
    - `start_ride_price` — стоимость начала поездки.
    - `subscription_fee` — ежемесячная плата за подписку.

In [3]:
# Загрузим данные из датасетов
users = pd.read_csv('datasets/users_go.csv')
rides = pd.read_csv('datasets/rides_go.csv')
subscriptions = pd.read_csv('datasets/subscriptions_go.csv')

In [4]:
# Взглянем на первые строки датафрейма users
users.head()

,user_id,name,age,city,subscription_type
0,1,Кира,22,Тюмень,ultra
1,2,Станислав,31,Омск,ultra
2,3,Алексей,20,Москва,ultra
3,4,Константин,26,Ростов-на-Дону,ultra
4,5,Адель,28,Омск,ultra


In [5]:
# Взглянем на первые строки датафрейма rides
rides.head()

,user_id,distance,duration,date
0,1,4409.919140,25.599769,2021-01-01
1,1,2617.592153,15.816871,2021-01-18
2,1,754.159807,6.232113,2021-04-20
3,1,2694.783254,18.511000,2021-08-11
4,1,4028.687306,26.265803,2021-08-28


In [6]:
# Взглянем на первые строки датайрейма subscriptions
subscriptions.head()

,subscription_type,minute_price,start_ride_price,subscription_fee
0,free,8,50,0
1,ultra,6,0,199


In [7]:
# Изучим общую информацию о датафрейме users
users.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1565 entries, 0 to 1564
Data columns (total 5 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   user_id            1565 non-null   int64 
 1   name               1565 non-null   object
 2   age                1565 non-null   int64 
 3   city               1565 non-null   object
 4   subscription_type  1565 non-null   object
dtypes: int64(2), object(3)
memory usage: 61.3+ KB


In [8]:
# Изучим общую информацию о датафрейме rides
rides.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18068 entries, 0 to 18067
Data columns (total 4 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   user_id   18068 non-null  int64  
 1   distance  18068 non-null  float64
 2   duration  18068 non-null  float64
 3   date      18068 non-null  object 
dtypes: float64(2), int64(1), object(1)
memory usage: 564.8+ KB


In [9]:
# Изучим общую информацию о датафрейме subscriptions
subscriptions.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2 entries, 0 to 1
Data columns (total 4 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   subscription_type  2 non-null      object
 1   minute_price       2 non-null      int64 
 2   start_ride_price   2 non-null      int64 
 3   subscription_fee   2 non-null      int64 
dtypes: int64(3), object(1)
memory usage: 196.0+ bytes


После загрузки данных мы изучили структуру каждого из датасетов (`users`, `rides`, `subscriptions`) для лучшего понимания информации, с которой будем работать.

Датасет `users`:

- Содержит информацию о пользователях: уникальный идентификатор, имя, возраст, город и тип подписки.
- **Количество записей**: 1565.
- **Пропущенные значения**: Нет пропущенных значений.
- **Типы данных**: числовые (`int64`) для идентификатора и возраста, текстовые (`object`) для имени, города и типа подписки.

Датасет `rides`:

- Включает данные о поездках: идентификатор пользователя, пройденное расстояние, продолжительность поездки и дату.
- **Количество записей**: 18,068.
- **Пропущенные значения**: Нет пропущенных значений.
- **Типы данных**: числовые (`float64` для расстояния и продолжительности, `int64` для идентификатора), текстовый формат для даты (`object`), который требует преобразования для удобства временного анализа.

Датасет `subscriptions`:

- Содержит условия подписки, включая тип подписки, стоимость минуты, стоимость начала поездки и ежемесячную плату.
- **Количество записей**: 2, так как в сервисе доступны только два типа подписки (free и ultra).
- **Пропущенные значения**: Нет пропущенных значений.
- **Типы данных**: числовые (`int64`) для стоимости минуты, стоимости начала поездки и ежемесячной платы, текстовый формат (`object`) для типа подписки.

**Общие наблюдения**

- Датасет `rides` включает большое количество записей.
- Формат даты в `rides` требует преобразования для дальнейшего анализа.
- Данные в `subscriptions` статичны (только 2 типа подписки), что упрощает анализ и использование информации при расчетах выручки.

## Предобработка данных

### Приведение типов

In [10]:
# Преобразуем тип колонки date датафрейма rides в datetime
rides['date'] = pd.to_datetime(rides['date'], format='%Y-%m-%d')

In [11]:
# Проверим результат
rides.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18068 entries, 0 to 18067
Data columns (total 4 columns):
 #   Column    Non-Null Count  Dtype         
---  ------    --------------  -----         
 0   user_id   18068 non-null  int64         
 1   distance  18068 non-null  float64       
 2   duration  18068 non-null  float64       
 3   date      18068 non-null  datetime64[ns]
dtypes: datetime64[ns](1), float64(2), int64(1)
memory usage: 564.8 KB


Создадим новый столбец с номером месяца на основе столбца date.

In [12]:
# Проверим уникальные годы в данных
rides['year'] = rides['date'].dt.year  # Создаем столбец с годом
unique_years = rides['year'].unique()
print("Уникальные годы в данных:", unique_years)

Уникальные годы в данных: [2021]


В датасете представлены данные только за один год - 2021. Мы можем безопасно использовать столбец `month` для анализа помесячных тенденций.

In [13]:
# Добавим новый столбец содержащий информацию о месяце
rides['month'] = rides['date'].dt.month

In [14]:
# Проверим результат
rides.head()

,user_id,distance,duration,date,year,month
0,1,4409.919140,25.599769,2021-01-01,2021,1
1,1,2617.592153,15.816871,2021-01-18,2021,1
2,1,754.159807,6.232113,2021-04-20,2021,4
3,1,2694.783254,18.511000,2021-08-11,2021,8
4,1,4028.687306,26.265803,2021-08-28,2021,8


### Обработка пропусков

In [15]:
# Проверим наличие пропусков в датафрейме users
users.isna().sum()

user_id              0
name                 0
age                  0
city                 0
subscription_type    0
dtype: int64

In [16]:
# Проверим наличие пропусков в датафрейме rides
rides.isna().sum()

user_id     0
distance    0
duration    0
date        0
year        0
month       0
dtype: int64

In [17]:
# Проверим наоичие пропусков в датафрейме subscriptions
subscriptions.isna().sum()

subscription_type    0
minute_price         0
start_ride_price     0
subscription_fee     0
dtype: int64

Ни в одном из датафреймов не было обнаружено пропущенных значений.

### Обработка дубликатов

In [19]:
# Проверим наличие дубликатов в датафреймах
print(f'Количество дубликатов в датафрейме users: {users.duplicated().sum()}')
print(f'Количество дубликатов в датафрейме rides: {rides.duplicated().sum()}')
print(f'Количество дубликатов в датафрейме subscriptions: {subscriptions.duplicated().sum()}')

Количество дубликатов в датафрейме users: 31
Количество дубликатов в датафрейме rides: 0
Количество дубликатов в датафрейме subscriptions: 0


Дубликаты присустствуют только в датафрейме users. Избавимся от явных дубликатов в этом датасете

In [20]:
# Избавимся от явных дубликатов в датафрейме users
users = users.drop_duplicates()

In [21]:
# Проверим результат
users.duplicated().sum()

0

На этом шаге мы выполнили основные пункты предобработки данных:
- Сконвертировали столбец date в датафрейме rides в datetime
- Добавили новую колонку с порядковым номером месяца
- Проверили наличие пропусков и дубликатов. Избавились от явных дубликатов в данных.